## 학습정리

### 22장. 네트워크 분석
* 네트워크(network) : 노드(node)와 그 사이를 연결하는 엣지(edge)로 구성
    * ex. 노드 - 엣지
        * 사용자 - 친구관계 : 방향이 없다(undirected)
        * 웹페이지 - 하이퍼링크 : 방향성 네트워크(directed network)
        
####  22.1 매개 중심성
* 연결 중심성(degree centrality) : 핵심인물이 직관이 일치하지 않는 경우 발생
* 매개 중심성(betweenness centality) : 임의의 두 사람 사이의 최단 경로상에 얼마나 자주 등장하는지를 보는 지수
    * 노드 i의 매개 중심성 : 다른 모든 노드 j,k쌍 사이의 최단 경로 중에, i를 거치는 경로의 비율로 계산
    * ex. Thor의 매개 중심성을 구하려면 Thor가 아닌 다른 모든 사람간의 최단 경로를 구해야함
        * 그 중 어떤 경로들이 Thor를 거쳐가는지 확인 
    * 최단 경로 : 임의의 두사람이 주어졌을 때 최단경로를 구할 수 있어야 함 
        * 너비 우선 탐색(breadth-first search) 적용 
            1. from_user를 받아서 다른 모든 사용자까지의 최단 경로를 계산해주는 함수를 생성
            2. 각 경로는 사용자 ID의 리스트로 표현, 리스트의 길이는 경로의 길이를 나타냄 
            3. shortest_paths_to : 사용자 ID를 키(key)로 사용하고, 해당 사용자에 도달할 때까지 경로를 나타내는 리스트를 값으로 사용하는 딕셔너리 
            4. frontier : 살펴보고 싶은 사용자의 순서를 볼때 사용하기위한 큐(queue)
                * 큐 : 항목을 추가 할 때는 뒤에, 제거할 때는 앞에서부터 제거하는 자료 구조 
            5. 네트워크를 살펴보면서 최단 경로를 모르는 새로운 이웃이 보이면 현재 사용자를 prev_user로 설정, 새로운 이웃을 큐에 추가
            6. 특정 사용자까지의 최단 경로를 계산하지 않더라도 사용자가 큐에서 제거되면 해당 사용자에 도달할 수 있는 최단 경로를 찾았다는 것을 알 수 있음 
            7. 큐에서 사용자를 제거할 때 해당 사용자까지의 최단 경로를 이미 계산 했다면, 새로운 경로를 찾았다는 것을 의미. 찾은 경로가 최단일 때 추가.
            8. 만약 큐에 더 이상 사용자가 남아있지 않다면 네트워크 전체를 살펴 봤다는 것을 의미. 종료
    * ex) 0,9가 다른 사용자 사이의 최단 경로 위에 존재 하지 않는 다면 매개 중심성은 0
        * 최단 경로 상에 빈번하게 위치할 경우 높은 매개 중심성을 가짐 
* 근접 중심성(closeness centrality) : 원접성(farness)을 이용해 구할수 있음 
    * 원접성(farness) : 어떤 사용자와 다른 모든 사용자의 최단 경로를 합한 값
    * 근접 중심성의 편차는 훨씬 적음 : 네트워크 중심에 있는 노드도 외곽에 위치한 노드들로부터 멀리 떨어져 있기 때문
* 일반 적으로는 쉽게 계산 가능한 고유벡터 중심성(eigenvector centrality)를 자주 사용 

####  22.2 고유 벡터 중심성
* 행렬 곱셈
    * 내적
        * m차원 벡터를 (m,1)행렬로 생각 할때, 이를 (n,m) 행렬과 곱하여 n차원 벡터로 간주할 수 있는 (n,1)행렬을 얻을 수 있음
        * (n,m)행렬을,m차원 벡터를 n차원 벡터로 변환하는 선형사상(linear mapping)
    * 고유벡터(eigenvector) : $Av = \lambda v$를 만족하는 벡터 $v$ 
    * 고윳값(eigenvalue) : $\lambda$값 
        * 고유 벡터를 찾는 방법 : 임의의 벡터 v를 골라 matrix_times_vector를 수행, 결과값의 크기가 1이 되게 재조정하는 과정을 반복 수행 

    
* 중심성

####  22.3 방향성 그래프와 페이지랭크
####  22.4 더 공부해 보고 싶다면

## code

In [11]:
# NamedTuple을 사용해서 데이터를 다루기 
from typing import NamedTuple

class User(NamedTuple) :
    id : int
    name : str

users = [User(0,"Hero"), User(1,"Dunn"), User(2,"Sue"), User(3,"Chi"),
         User(4,"Thor"), User(5,"Clive"), User(6,"Hicks"), User(7,"Devin"), User(8,"Kate"), User(9,"Klein")]

friend_pairs = [(0,1),(0,2),(1,2),(1,3),(2,3),(3,4),(4,5),(5,6),(5,7),(6,8),(7,8),(8,9)]


In [12]:
from typing import Dict, List

# 친구 관계를 유지하기 위한 type alias
Friendships = Dict[int, List[int]]

friendships : Friendships = {user.id : [] for user in users}

for i,j in friend_pairs:
    friendships[i].append(j)
    friendships[j].append(i)
    

In [13]:
friendships[4]

[3, 5]

In [14]:
# 매개 중심성을 구하기 위한 최단거리 모델
# 너비 우선 탐색 
from collections import deque

Path = List[int]

def shortest_paths_from(from_user_id: int, friendships: Friendships) -> Dict[int, List[Path]] :
    # 특정 사용자로부터 다른 사용자까지의 모든 최단 경로를 포함하는 dict 
    shortest_paths_to : Dict[int, List[Path]] = {from_user_id: [[]]}
    
    # 확인해야 하는 (이전 사용자, 다음사용자) 큐
    # 모든 (from_user, from_user의 친구) 쌍으로 시작
    frontier = deque((from_user_id, friend_id) for friend_id in friendships[from_user_id])
    
    # 큐가 빌 때까지 반복
    while frontier :
        # 큐의 첫 번째 사용자를 제거
        prev_user_id, user_id = frontier.popleft()
        
        # 큐에 사용자를 추가하는 방법을 고려해 보면 
        # prev_user까지의 최단 경로의 몇 개를 이미 알고 있다.
        paths_to_prev_user = shortest_paths_to[prev_user_id]
        new_paths_to_user = [path + [user_id] for path in paths_to_prev_user]
        
        # 만약 최단 경로를 이미 알고 있다면 
        old_paths_to_user = shortest_paths_to.get(user_id, [])
        
        # 지금까지의 최단 경로는 무엇일까?
        if old_paths_to_user :
            min_path_length = len(old_paths_to_user[0])
        else :
            min_path_length = float('inf')
        
        # 길지 않은 새로운 경로만 저장
        new_paths_to_user = [path for path in new_paths_to_user 
                             if len(path) <= min_path_length and path not in old_paths_to_user]
        
        shortest_paths_to[user_id] = old_paths_to_user + new_paths_to_user
        
        # 아직 한 번도 보지 못한 이웃을 frontier에 추가
        frontier.extend((user_id, friend_id) for friend_id in friendships[user_id]
                       if friend_id not in shortest_paths_to)
        
        return shortest_paths_to
            

In [15]:
# 최단거리 계산
# 각 from_user에서 각 to_user까지 최단 경로 리스트를 반환
shortest_paths = {user.id: shortest_paths_from(user.id, friendships) for user in users}


In [16]:
# 최단 경로에 포함되는 각 노드의 매개 중심성에 1/n을 더해줌

betweenness_centrality = {user.id: 0.0 for user in users}

for source in users :
    for target_id, paths in shortest_paths[source.id].items() :
        if source.id < target_id :               # 잘못해서 두 번 세지 않도록 주의
            num_paths = len(paths)               # 최단 경로가 몇개 존재?
            contrib = 1/num_paths                # 중심성에 기여하는 값
            for path in paths :
                for between_id in path :
                    if between_id not in [source.id, target_id] :
                        betweenness_centrality[between_id] += contrib

In [17]:
betweenness_centrality

{0: 0.0,
 1: 0.0,
 2: 0.0,
 3: 0.0,
 4: 0.0,
 5: 0.0,
 6: 0.0,
 7: 0.0,
 8: 0.0,
 9: 0.0}

In [18]:
# 원접성 
def farness(user_id: int) -> float :
    """모든 사용자와의 최단 거리 합"""
    return sum(len(paths[0]) for paths in shortest_paths[user_id].values())

In [22]:
closeness_centrality = {user.id : 1 / farness(user.id) for user in users}
closeness_centrality

{0: 1.0,
 1: 1.0,
 2: 1.0,
 3: 1.0,
 4: 1.0,
 5: 1.0,
 6: 1.0,
 7: 1.0,
 8: 1.0,
 9: 1.0}

In [4]:
# 내적
from typing import Callable, Tuple, List

Matrix = List[List[float]]

def make_matrix(num_rows : int,
               num_cols : int,
               entry_fn : Callable[[int,int],float]) -> Matrix :
    """
    (i,j)번쨰 원소가 entry_fn(i,j)인 num_row x num_cols 리스트를 반환
    """
    return [[entry_fn(i,j)
            for j in range(num_cols)]
           for i in range(num_rows)]

def shape(A : Matrix) -> Tuple[int, int] :
    """(행의 개수,열의 개수)반환"""
    num_rows = len(A)
    num_cols = len(A[0]) if A else 0 # 첫 번째 행의 원소의 개수
    return num_rows, num_cols

In [5]:
def matrix_times_matrix(m1: Matrix, m2: Matrix) -> Matrix :
    nr1, nc2 = shape(m1)
    nr2, nc2 = shape(m2)
    assert nc1 == nr2, "must have (# of columns in m1) == (# of rows in m2)"
    
    def entry_fn(i : int, j: int) -> float :
        """m1의 i번째 행과 m2의 j번째 열의 내적 """
        return sum(m1[i][k] * m2[k][j] for k in range(nc1))
    
    return make_matrix(nr1, nc2, entry_fn)

In [7]:
# 선형 사상

Vector = List[float]


def matrix_times_vector(m : Matrix, v: Vector) -> Vector :
    nr,nc = shape(m)
    n = len(v)
    assert nc == n, "must have (# of cols in m) == (# of elements in v)"
    
    return [np.dot(row,v) for row in m] # output has length nr 


In [9]:
# 고유벡터 
from typing import Tuple
import random
# from scratch.linear_algebra import magnitude,distance
import math # math.sqrt : 제곱근

def magnitude(v:Vector) -> float :
    """벡터 v의 크기를 반환"""
    return math.sqrt(sum_of_squares(v))

def squared_distance(v: Vector, w: Vector) -> float :
    """(v_1 - w_1) ** 2 +...+ (v_n - w_n) ** 2 """
    return sum_of_squares(subtract(v,w))

def distance(v: Vector, w: Vector) -> float :
    """벡터 v와 w간의 거리를 계산"""
    return math.sqrt(squared_distance(v,w))




def find_eigenvector(m : Matrix, tolerance : float =0.00001) -> Tuple[Vector, float] :
    guess = [random.random() for _ in m]
    
    while True :
        result = matrix_times_vector(m, guess) # guess 반환
        norm = magnitude(result)               # 크기를 계산
        next_guess = [x / norm for x in result] # 재조정
        
        if distance(guess, next_guess) < tolerance :
            # 수렴했으니 (고유벡터, 고윳값)을 반환
            return next_guess, norm 
        
        guess = next_guess

SyntaxError: invalid syntax (<ipython-input-9-0107cc30faf6>, line 30)